In [1]:
#Import Modules
import twobitreader as tbr
import time
import multiprocessing as mp
import pandas as pd
import numpy as np

In [2]:
#Opening human genome file into memory
gen_file = "/Users/omarkana/Documents/sudin_pipeline/data/hg19.2bit"
genome = tbr.TwoBitFile(gen_file)

In [11]:
#To test speed of non-parallel functions
chr1 = genome['chr1'][:10000000]
chr1 = chr1.lower()
len(chr1)

10000000

In [12]:
#Overlapping
def GetGenomeVocab(dna, min_chunk_size, max_chunk_size):
    vocab = []
    for n in range(min_chunk_size, max_chunk_size, 1):
        vocab = list(set(vocab))
        x = [dna[i:i+n] for i in range(len(dna))]
        vocab.extend(list(set(x)))
    return list(set(vocab))
#Non-overlapping
#Non-overlapping seems to be infeasible on DNA strings orders higher than 1e5. Thus it is difficult to produce a sizable library. If we are looking at fixed BP lengths then this is a useful option
def NoDnaVocab(dna, mini, maxi):
    vocab = []
    for m in range(len(dna)):
        for n in range(mini, maxi, 1):
            vocab = list(set(vocab))
            x = [dna[i:i+n] for i in range(m, len(dna), n)]
            vocab.extend(list(set(x)))
    return list(set(vocab))
#Parallel for DHS
def OGenomeVocabmp(dna_list, mini, maxi, workers = 1):
    pool = mp.Pool(processes=workers)
    results = [pool.apply(GetGenomeVocab(d, min_chunk_size = mini, max_chunk_size = maxi)) for d in dna_list]
    return list(set(results))


In [13]:
#For meaningful Overlapping Sequences
start_time = time.time()
vocab = GetGenomeVocab(chr1,11,18) 
print("--- %s seconds ---" % (time.time() - start_time))
print(len(vocab))

--- 74.01625275611877 seconds ---
46445402


In [ ]:
#For a meaningful non-overlapping sequneces
vocab = []
start_time = time.time()
vocab = NoDnaVocab(chr1, 3, 15)
print("--- %s seconds ---" % (time.time() - start_time))

# Length of all DHS regions

In [15]:
dhs_table = pd.read_table('../data/allGeneCorrelations100000.p05_v3.txt')

In [16]:
diff = np.subtract(dhs_table['dhs_end'].values, dhs_table['dhs_start'].values)
total = np.sum(diff)
print(total)

119258200


In [17]:
dhs_loc = dhs_table[['dhs_chr', 'dhs_start', 'dhs_end']]

In [55]:
#Gather subset of DHS sites
dhs_seq = []
for i in dhs_loc.values:
    x = genome[i[0]][i[1]:i[2]].upper()
    dhs_seq.append(x)

In [ ]:
#Gathering Vocabulary
start_time = time.time()
vocab = []
for i in dhs_seq:
    x = GetGenomeVocab(i, 3, 15)
    vocab.extend(list(set(x)))
vocab = list(set(vocab))
with open('../data/dhs_vocab.txt', 'w') as f:
    for item in vocab:
        f.write("%s\n" % item)
vocab = []
print("--- %s seconds ---" % (time.time() - start_time))

In [53]:
start_time = time.time()
vocab = []
def ochunker(dna, c_size):
    c_size = n
    x = [dna[i:i+n] for i in range(0, len(dna), n)]
    return x
if __name__ == '__main__':
    # start 4 worker processes
    with mp.Pool(processes=8) as pool:
        for n in range(3, 15, 1):
            vocab = [pool.apply(ochunker, args=(d, n)) for d in dhs_seq]
print("--- %s seconds ---" % (time.time() - start_time))

--- 19.228724002838135 seconds ---


In [14]:
vocab

['catcagcagatttc',
 'ttcataaacaatgctt',
 'aatattatgtccctcat',
 'ctgctcccatgtggg',
 'ctccagaccagct',
 'tgggccatgtccaggtg',
 'ggaaaatcattt',
 'atacaggatcatgt',
 'catctgcgggcgac',
 'aatacagacacggagca',
 'tgtagtcctatcccagc',
 'agtcaacaaactgaa',
 'gtatgcactattcaa',
 'gttccagaacatctca',
 'aaacactgtctt',
 'ctctatccttccttcct',
 'tggccacacggg',
 'agcctgagcagagacaa',
 'tacttatttac',
 'ggcgcatccactc',
 'attgggcccagttag',
 'tctggacgcagggccct',
 'gggtgcaaaagtgcagt',
 'cattcactgtttattt',
 'gcactgggccaggtg',
 'cactcccttcaatct',
 'tctacaagcctctt',
 'cgagggcatgattct',
 'acggtcttggg',
 'ttttgtgaatcc',
 'cccatactcacatctg',
 'tagaatgttctacaaat',
 'tcgcctctgttc',
 'tgagactggagt',
 'actcatggaactga',
 'ggggtcacacaaaaaga',
 'cattgtccagctcctcc',
 'gacgtatttaaata',
 'cttcgaggagcac',
 'ccttttaaaaaacata',
 'cacccttgaaag',
 'gtttccttctgttt',
 'gttaaatgatgagt',
 'agcctaatgcag',
 'cgaatcgctgaaa',
 'gccccctcgaa',
 'ctaagagagcaa',
 'ttgtggagtcac',
 'tatatttgcat',
 'tctggagaaacct',
 'gttggggtcgtgg',
 'actgggtctcctc',
 

In [2]:
len('GDIFYPGYCPDVKPVNDFDLSAFAGAWHEIAKLPLENENQGKCTIAEYKY')

50